In [ ]:
!pip install --quiet langgraph "langchain>=0.2.13,<0.3.0" langchain-aws opensearch-py

In [ ]:
!pip install --upgrade pydantic --quiet

In [ ]:
import os
import boto3
from random import randint

ssm_client = boto3.client('ssm')

response = ssm_client.get_parameters(
    Names=[
        'AOSSCollectionName', 'AOSSEmbeddingSize', 'AOSSHost', 'AOSSIndexName', 'S3BucketName'
    ]
)
param_dict = {}
for parameter in response['Parameters']:
    param_dict[parameter['Name']] = parameter['Value']

os.environ["region"] = param_dict['AOSSHost'].split(".")[1]
os.environ["index_name"] = param_dict['AOSSIndexName']
os.environ["collection_name"] = param_dict['AOSSCollectionName']
os.environ["aoss_host"] = param_dict['AOSSHost']
os.environ["s3_bucket"] = param_dict['S3BucketName']
os.environ["embeddingSize"] = param_dict['AOSSEmbeddingSize']

In [ ]:
import io
from random import randint
from helpers import download_from_s3
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage, AIMessage

from graph import graph

In [ ]:
RECURSION_LIMIT = 10

In [ ]:
def add_system_prompt(messages):
    system_prompt = """You are a smart and quirky AI Stylist. You answer questions about clothes to wear. You have to understand the user question and give precise answer to the user. <Instructions> 1/ Try to take weather and occasion into your suggestions.  2/ If you do not find any relevant image in the database, generate an image using your tools. 3/ Try to respond the user with a relevant image."""
    return [SystemMessage(content=system_prompt)] + messages


def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        print(message)
        if isinstance(message, tuple):
            print(message.content)
        else:
            message.pretty_print()

### Define User Question

In [ ]:
USER_QUERY = "Can you find me an image with dotted shirt and dark blue jeans?"


input_message = add_system_prompt([HumanMessage(content=USER_QUERY)])
input_message

In [ ]:
thread_id = randint(0, 9999)
s3_path = ""
for out in graph.stream(
    {"messages": input_message},
    config={
        "recursion_limit": RECURSION_LIMIT,
        "configurable": {"thread_id": thread_id}},
        stream_mode="values"):
    msg = next(iter(out.values()))
    last_msg = msg[-1]
    if isinstance(last_msg, ToolMessage):
        if last_msg.content.startswith("s3"):
            s3_path = last_msg.content
            print(f"\nToolMessage: S3 Path for output image: {last_msg.content}")
        else:
            print(f"\nToolMessage: {last_msg.content}")
    elif isinstance(last_msg, AIMessage):
        if isinstance(last_msg.content,list) and last_msg.content[0]["type"]=="tool_use":
            print(f"\nCalling a tool.. {last_msg.content[0]['name']} with input {last_msg.content[0]['input']}")
        print("\nAIMessage:")
        print(last_msg.content)

In [ ]:
s3_key = s3_path.replace("s3://"+os.environ["s3_bucket"]+"/", '')
s3_key

In [ ]:
download_from_s3(os.environ["s3_bucket"], key=s3_key)

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass